In [54]:
import pandas as pd
import numpy as np


import transformers
from transformers import pipeline
import torch

In [55]:
path_file = "C:\\Users\\daniel.martinezcarre\\Desktop\\Repositorios\\ia_repositorio\\sistemas_de_aprendizaxe_automatico\\neumonia\\peruvia-food-reviews\\reviews.csv"

data = pd.read_csv(path_file)

data.columns

Index(['id_review', 'review', 'title', 'score', 'likes', 'id_nick', 'service',
       'date', 'platform'],
      dtype='object')

### Se recogen sólo los reviews del dataframe

In [56]:
data_modelo = data[:1000]
data_modelo

,id_review,review,title,score,likes,id_nick,service,date,platform
0,R1245,Muy buena presentación y servicio sin embargo ...,Muy buena presentación y servicio,3.0,0,uisahelenad2018,71036.0,1 years ago,tripadvisor
1,R1246,Desde la presentación de los platos a la calid...,Una experiencia,5.0,0,turrutia,71036.0,1 years ago,tripadvisor
2,R1247,El mejor lugar para reencontrame con mis amigo...,Felicitaciones a Statera!,5.0,0,orthStar25333985745,71036.0,1 years ago,tripadvisor
3,R1248,Excelente experiencia Comida maravillosa con e...,Aniversario de boda,5.0,0,atima J,71036.0,1 years ago,tripadvisor
4,R1249,Mi pasión es viajar y disfrutar de las grandez...,No te lo querrás perder!,5.0,0,nbocadoalmundo,71036.0,1 years ago,tripadvisor
...,...,...,...,...,...,...,...,...,...
995,R3876,Sitio agradable y buena atención. Las hamburgu...,Buena hamburguesa,5.0,0,uancU4383RV,104517.0,5 years ago,tripadvisor
996,R3877,Muy buena idea de ponerle queso a la salchicha...,Recomendable !,5.0,0,esús S,104517.0,5 years ago,tripadvisor
997,R3878,Es la tercera vez que voy a JL porque sus hamb...,Cuidado con la preparación,3.0,0,agopaul,104517.0,5 years ago,tripadvisor
998,R3879,Pasamos por aquí luego de hacer unas compras y...,Muy buena sazón,4.0,0,eliGP,104517.0,5 years ago,tripadvisor


## Comprobar si hay una GPU disponible

In [57]:
device = 0 if torch.cuda.is_available() else -1
device

-1

In [58]:
# https://huggingface.co/docs/transformers/main_classes/logging
transformers.utils.logging.set_verbosity_error()

## Análisis sentimiento

In [59]:
from transformers import pipeline

# Se carga el modelo pre-entrenado de BERT para análisis de sentimiento en ESPAÑOL
nlp = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)

In [60]:
len(data_modelo)

1000

In [61]:
from transformers import BertTokenizer

# Carga el tokenizador BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokeniza las reseñas
tokenized_reviews = []
for review in data_modelo:
    encoded_inputs = tokenizer(review, truncation=True, padding='max_length', max_length=800)
    tokenized_reviews.append(encoded_inputs)

### Aplicación del análisis de sentimiento a cada review

In [62]:
len(tokenized_reviews)

9

In [63]:
resultados = []
for review in tokenized_reviews:
    tokens_str = [str(token) for token in review['input_ids']]
    result = nlp(tokens_str)[0]
    resultados.append(result)

### Agregar los sentimientos al dataframe
Se crean 2 nuevas columnas en el dataframe para almacenar la etiqueta de sentimiento (positivo, negativo o neutral) y la puntuación asociada a cada reseña.

In [65]:
sentimientos = []
puntuaciones = []

for resultado, fila in zip(resultados, data_modelo.iterrows()):
  # Relleno con valores predeterminados si la longitud del resultado es menor
  if len(resultado) < len(data_modelo):
    sentimientos.extend(["NEUTRAL"] * (len(data_modelo) - len(resultado)))
    puntuaciones.extend([0.0] * (len(data_modelo) - len(resultado)))
  else:
    sentimientos.extend([resultado['label'] for resultado in resultados])
    puntuaciones.extend([resultado['score'] for resultado in resultados])

data_modelo['sentimiento'] = sentimientos
data_modelo['puntuacion_sentimiento'] = puntuaciones


ValueError: Length of values (8982) does not match length of index (1000)

In [ ]:
data_modelo